In [16]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [17]:
raw_dir = '/workspace/COMP3610-Renewable-Energy-Prediction/data/raw'

In [18]:
# Ensure processed directory exists
processed_dir = '../data/processed'
os.makedirs(processed_dir, exist_ok=True)

## Create a Data Loading Function

In [19]:
def load_country_data(country):
    """Load raw data for a specific country"""
    file_path = f'../data/raw/{country}_Power_Generation.csv'
    df = pd.read_csv(file_path)
    print(f"Loaded {country} data with shape: {df.shape}")
    return df

## Create Functions to Clean each Dataset

In [20]:
def clean_france_data(df):
    # Handle 'n/e' values in object columns (convert to NaN then handle)
    object_cols = df.select_dtypes(include='object').columns[4:]  # Skip Area, MTU, DATETIME
    df[object_cols] = df[object_cols].replace('n/e', np.nan)
    
    # Convert object columns that should be numeric
    for col in object_cols:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            # For columns that are truly categorical (like Nuclear which is all 'n/e')
            df = df.drop(col, axis=1)
    
    # Parse datetime properly
    df['DATETIME'] = pd.to_datetime(df['DATETIME'], format='%d.%m.%Y %H:%M')
    df['MTU_start'] = pd.to_datetime(df['MTU'].str.split(' - ').str[0], format='%d.%m.%Y %H:%M')
    
    # Handle missing values - different strategies per column
    # For columns with <5% missing (most columns)
    cols_under_5 = [col for col in df.columns if df[col].isnull().mean() < 0.05 and df[col].dtype in ['float64', 'int64']]
    df[cols_under_5] = df[cols_under_5].interpolate(method='time')
    
    # For columns with high missingness (Hydro Pumped Storage)
    cols_over_50 = ['Hydro Pumped Storage  - Actual Aggregated [MW]', 
                   'Hydro Pumped Storage  - Actual Consumption [MW]']
    df[cols_over_50] = df[cols_over_50].fillna(0)  # Assuming no pumping when data missing
    
    # Feature engineering
    df['hour'] = df['DATETIME'].dt.hour
    df['day_of_week'] = df['DATETIME'].dt.dayofweek
    df['month'] = df['DATETIME'].dt.month
    df['season'] = (df['month'] % 12 + 3) // 3
    
    # 6. Calculate total renewable energy (focus of our prediction)
    renewable_cols = ['Solar  - Actual Aggregated [MW]', 
                     'Wind Onshore  - Actual Aggregated [MW]',
                     'Wind Offshore  - Actual Aggregated [MW]',
                     'Hydro Run-of-river and poundage  - Actual Aggregated [MW]',
                     'Hydro Water Reservoir  - Actual Aggregated [MW]']
    
    df['Total_Renewable'] = df[renewable_cols].sum(axis=1)
    
    # 7. Select and rename columns
    columns_to_keep = [
        "Area", 
        "MTU", 
        "DATETIME", 
        "YEAR", 
        "Solar  - Actual Aggregated [MW]", 
        "Wind Offshore  - Actual Aggregated [MW]", 
        "Wind Onshore  - Actual Aggregated [MW]"
    ]

    # Select all columns except the ones to keep
    columns_to_drop = [col for col in df.columns if col not in columns_to_keep]

    # Drop those columns
    df = df.drop(columns=columns_to_drop)
    
    return df

In [21]:
def clean_italy_data(df):
    # 1. Handle 'n/e' values
    object_cols = df.select_dtypes(include='object').columns[4:]
    df[object_cols] = df[object_cols].replace('n/e', np.nan)
    
    # 2. Nuclear is all 'n/e' - drop
    df = df.drop('Nuclear  - Actual Aggregated [MW]', axis=1)
    
    # 3. Parse datetime
    df['DATETIME'] = pd.to_datetime(df['DATETIME'], format='%d.%m.%Y %H:%M')
    
    # 4. Handle missing values
    # Fossil Coal-derived gas has 575 missing - inspect pattern
    if df['Fossil Coal-derived gas  - Actual Aggregated [MW]'].isnull().sum() > 0:
        df['Fossil Coal-derived gas  - Actual Aggregated [MW]'] = df['Fossil Coal-derived gas  - Actual Aggregated [MW]'].fillna(0)
    
    # Other columns with minimal missingness
    cols_under_5 = [col for col in df.columns if df[col].isnull().mean() < 0.05 and pd.api.types.is_numeric_dtype(df[col])]
    df[cols_under_5] = df[cols_under_5].interpolate(method='time')
    
    # High missingness columns
    df['Hydro Pumped Storage  - Actual Consumption [MW]'] = df['Hydro Pumped Storage  - Actual Consumption [MW]'].fillna(0)
    
    # 5. Feature engineering
    df['hour'] = df['DATETIME'].dt.hour
    df['day_of_week'] = df['DATETIME'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)
    
    # 6. Total renewable calculation
    renewable_cols = ['Solar  - Actual Aggregated [MW]',
                    'Wind Onshore  - Actual Aggregated [MW]',
                    'Geothermal  - Actual Aggregated [MW]',
                    'Hydro Run-of-river and poundage  - Actual Aggregated [MW]']
    
    df['Total_Renewable'] = df[renewable_cols].sum(axis=1)
    
    # 7. Select and rename columns
    columns_to_keep = [
        "Area", 
        "MTU", 
        "DATETIME", 
        "YEAR", 
        "Solar  - Actual Aggregated [MW]", 
        "Wind Offshore  - Actual Aggregated [MW]", 
        "Wind Onshore  - Actual Aggregated [MW]"
    ]

    # Select all columns except the ones to keep
    columns_to_drop = [col for col in df.columns if col not in columns_to_keep]

    # Drop those columns
    df = df.drop(columns=columns_to_drop)
    
    return df

In [22]:
def clean_spain_data(df):
    # Spain data is mostly numeric already
    
    # 1. Parse datetime
    df['DATETIME'] = pd.to_datetime(df['DATETIME'], format='%d.%m.%Y %H:%M')
    
    # 2. Handle missing values (mostly minimal)
    cols_with_missing = [col for col in df.columns if df[col].isnull().sum() > 0 and pd.api.types.is_numeric_dtype(df[col])]
    for col in cols_with_missing:
        if df[col].isnull().mean() < 0.01:  # Less than 1% missing
            df[col] = df[col].interpolate(method='time')
        else:
            df[col] = df[col].fillna(0)  # For columns like Marine which are mostly 0
    
    # 3. Feature engineering
    df['hour'] = df['DATETIME'].dt.hour
    df['month'] = df['DATETIME'].dt.month
    df['season'] = (df['month'] % 12 + 3) // 3
    
    # 4. Total renewable calculation
    renewable_cols = ['Solar  - Actual Aggregated [MW]',
                     'Wind Onshore  - Actual Aggregated [MW]',
                     'Wind Offshore  - Actual Aggregated [MW]',
                     'Hydro Run-of-river and poundage  - Actual Aggregated [MW]',
                     'Hydro Water Reservoir  - Actual Aggregated [MW]']
    
    df['Total_Renewable'] = df[renewable_cols].sum(axis=1)
    
    # 7. Select and rename columns
    columns_to_keep = [
        "Area", 
        "MTU", 
        "DATETIME", 
        "YEAR", 
        "Solar  - Actual Aggregated [MW]", 
        "Wind Offshore  - Actual Aggregated [MW]", 
        "Wind Onshore  - Actual Aggregated [MW]"
    ]

    # Select all columns except the ones to keep
    columns_to_drop = [col for col in df.columns if col not in columns_to_keep]

    # Drop those columns
    df = df.drop(columns=columns_to_drop)
    
    return df

## Process Each Country's Data

In [ ]:
def clean_france_data(df):
    # 1. First convert DATETIME to proper datetime and set as index
    df['DATETIME'] = pd.to_datetime(df['DATETIME'], format='%d.%m.%Y %H:%M')
    df = df.set_index('DATETIME')
    
    # 2. Handle 'n/e' values in object columns (convert to NaN then handle)
    object_cols = df.select_dtypes(include='object').columns[3:]  # Skip Area, MTU, YEAR
    df[object_cols] = df[object_cols].replace('n/e', np.nan)
    
    # 3. Convert object columns that should be numeric
    for col in object_cols:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            # For columns that are truly categorical
            df = df.drop(col, axis=1)
    
    # 4. Now handle missing values with time interpolation
    cols_under_5 = [col for col in df.columns 
                   if df[col].isnull().mean() < 0.05 
                   and pd.api.types.is_numeric_dtype(df[col])]
    
    # Only interpolate if we have datetime index
    if isinstance(df.index, pd.DatetimeIndex):
        df[cols_under_5] = df[cols_under_5].interpolate(method='time')
    else:
        df[cols_under_5] = df[cols_under_5].interpolate()
    
    # 5. Handle high missingness columns
    cols_over_50 = ['Hydro Pumped Storage  - Actual Aggregated [MW]', 
                   'Hydro Pumped Storage  - Actual Consumption [MW]']
    for col in cols_over_50:
        if col in df.columns:
            df[col] = df[col].fillna(0)
    
    # 6. Feature engineering
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['month'] = df.index.month
    df['season'] = (df['month'] % 12 + 3) // 3
    
    # 7. Calculate total renewable energy
    renewable_cols = ['Solar  - Actual Aggregated [MW]', 
                     'Wind Onshore  - Actual Aggregated [MW]',
                     'Wind Offshore  - Actual Aggregated [MW]',
                     'Hydro Run-of-river and poundage  - Actual Aggregated [MW]',
                     'Hydro Water Reservoir  - Actual Aggregated [MW]']
    
    df['Total_Renewable'] = df[renewable_cols].sum(axis=1)
    
    # 8. Drop unnecessary columns
    drop_cols = ['MTU', 'Area', 'Fossil Peat  - Actual Aggregated [MW]', 
                'Marine  - Actual Aggregated [MW]']
    df = df.drop(columns=[col for col in drop_cols if col in df.columns])
    
    return df.reset_index()  # Return with DATETIME as column again


Processing France data...


/tmp/ipykernel_2584/3655646536.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[object_cols] = df[object_cols].replace('n/e', np.nan)


ValueError: time-weighted interpolation only works on Series or DataFrames with a DatetimeIndex

## Create Combined Dataset

In [ ]:
# Combine all country data
combined_df = pd.concat(cleaned_data.values())

# Save combined data
combined_path = f'{processed_dir}/EU_Power_Generation_Combined_Cleaned.csv'
combined_df.to_csv(combined_path)
print(f"Saved combined cleaned data to {combined_path}")

Saved combined cleaned data to ../data/processed/EU_Power_Generation_Combined_Cleaned.csv
